In [ ]:
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# from utils.plot_utils import overlay_colormap
from datasets.generate_map_from_bbox_person import load_image, load_target
from utils.coco import COCO_ID_TO_CHANNEL

import numpy as np
from pycocotools.coco import COCO
from pathlib import Path
import cv2
import pprint
from collections import defaultdict

COCO_FOLDER = "/mnt/ssd2/xxx/data/coco/"

## pos_weight for cmap person
recommend to use the second method instead for flexibility


In [ ]:
# coco_folder = Path(COCO_FOLDER)
# split = "val2017"
# split = "train2017"
# root = coco_folder / split
# annFile = coco_folder / "annotations" / f"instances_{split}.json"
# mask_folder = coco_folder / "cmaps_person" / split

# # Load COCO dataset
# coco = COCO(annFile)
# ids = list(sorted(coco.imgs.keys()))

# with open(mask_folder.parent / (split + "-ids.txt"), "r") as f:
#     ids_person_str = f.read().splitlines()
#     # strip left zeros and to int
#     ids_person = list(map(int, ids_person_str))

# ids_other = list(set(ids) - set(ids_person))

In [ ]:
# res = {}
# area_all = 0
# area_centerness_all = 0
# area_centerness_50_all = 0

# for id in ids:
#     tmp = {}
#     img = load_image(coco, root, id)
#     w, h = img.size
#     area = w * h
#     tmp["area"] = area
#     tmp["area_centerness"] = 0
#     tmp["area_centerness_50"] = 0
#     res[id] = tmp
#     area_all += area


# for id_str in ids_person_str:
#     mask = (
#         cv2.imread(mask_folder / (id_str + ".png"), cv2.IMREAD_UNCHANGED).astype(
#             np.float32
#         )
#         / 255.0
#     )
#     area_centerness = np.sum(mask > 0)
#     area_centerness_50 = np.sum(mask > 0.5)

#     res[int(id_str)]["area_centerness"] = area_centerness
#     res[int(id_str)]["area_centerness_50"] = area_centerness_50
#     area_centerness_all += area_centerness
#     area_centerness_50_all += area_centerness_50

In [7]:
def generate_ids_file(directory):
    # List all files in the directory
    files = os.listdir(directory)

    # Filter out only the .png files and remove the extension to get the IDs
    ids = [os.path.splitext(file)[0] for file in files if file.endswith(".png")]

    # Sort the IDs numerically
    ids.sort()

    # Write the IDs to the output file
    output_file = directory + "-ids.txt"
    with open(output_file, "w") as f:
        for id in ids:
            f.write(id + "\n")

    print(f"{output_file} file has been created successfully.")


# # Define the paths to the directories
# train_dir = "train2017"
# val_dir = "val2017"

# # Generate train-ids.txt
# generate_ids_file(train_dir)

# # Generate val-ids.txt
# generate_ids_file(val_dir)

In [ ]:
def get_pos_weight_alpha(
    split="val2017_person",
    coco_folder=COCO_FOLDER,
    mask_folder="cmaps_person",
    probas=np.linspace(0, 1, 11)[:-1],
):
    print(f"Calculating pos_weight and alpha for {split}...")

    coco_folder = Path(coco_folder)
    annFile = coco_folder / "annotations" / f"instances_{split}.json"
    coco = COCO(annFile)

    split = split.split("_")[0]

    root = coco_folder / split
    mask_folder = coco_folder / mask_folder / split

    ids = list(sorted(coco.imgs.keys()))

    txt_path = mask_folder.parent / (split + "-ids.txt")
    if not txt_path.exists():
        generate_ids_file(str(mask_folder.absolute()))

    with open(txt_path, "r") as f:
        ids_person_str = f.read().splitlines()
        # # strip left zeros and to int
        # ids_person = list(map(int, ids_person_str))

    area_all = 0
    area_centerness_all = {proba: 0 for proba in probas}

    # for id in ids:
    #     img = load_image(coco, root, id)
    #     w, h = img.size
    #     area = w * h
    #     area_all += area

    for id_str in ids_person_str:
        # get rid of preceding zeros in id_str
        id = int(id_str)
        img = load_image(coco, root, id)
        w, h = img.size
        area = w * h
        area_all += area

        try:
            mask = (
                cv2.imread(
                    mask_folder / (id_str + ".png"), cv2.IMREAD_UNCHANGED
                ).astype(np.float32)
                / 255.0
            )
        except:
            print(f"Error loading {mask_folder / (id_str + '.png')}")
        for proba in probas:
            area_centerness_all[proba] += np.sum(mask > proba)

    pos_weight_all = {k: area_all / v for k, v in area_centerness_all.items()}
    alpha_all = {k: 1 - 1 / v for k, v in pos_weight_all.items()}

    print(f"\npos_weight_all for {split}:")
    pprint.pprint(pos_weight_all)

    print(f"\nalpha_all for {split}:")
    pprint.pprint(alpha_all)

    return pos_weight_all, alpha_all

In [12]:
pos_weight_all, alpha_all = get_pos_weight_alpha("val2017_person")

Calculating pos_weight and alpha for val2017_person...
loading annotations into memory...
Done (t=0.12s)
creating index...
index created!

pos_weight_all for val2017:
{0.0: 3.606717387392169,
 0.1: 4.1283694509697035,
 0.2: 5.282034376573399,
 0.30000000000000004: 7.1494939578340055,
 0.4: 10.446969868779867,
 0.5: 16.128149160971073,
 0.6000000000000001: 27.780025203336514,
 0.7000000000000001: 53.39112951806463,
 0.8: 133.78171998496938,
 0.9: 557.5448191029582}

alpha_all for val2017:
{0.0: 0.7227395738031339,
 0.1: 0.7577736169506069,
 0.2: 0.8106790057188671,
 0.30000000000000004: 0.8601299608199182,
 0.4: 0.9042784642283273,
 0.5: 0.9379966051889,
 0.6000000000000001: 0.9640029124278874,
 0.7000000000000001: 0.981270296975799,
 0.8: 0.9925251372152163,
 0.9: 0.998206422218022}


In [15]:
pos_weight_all, alpha_all = get_pos_weight_alpha(
    split="val2017_person", coco_folder=COCO_FOLDER, mask_folder="gmaps_kp_person"
)

Calculating pos_weight and alpha for val2017_person...
loading annotations into memory...
Done (t=0.12s)
creating index...
index created!

pos_weight_all for val2017:
{0.0: 505.50716464378934,
 0.1: 1133.5330843807433,
 0.2: 1867.7884588024876,
 0.30000000000000004: 2382.6298746123157,
 0.4: 3289.6089829924517,
 0.5: 3289.6089829924517,
 0.6000000000000001: 5312.959682606213,
 0.7000000000000001: 7673.270801152256,
 0.8: 13809.523686088884,
 0.9: 69032.05840375587}

alpha_all for val2017:
{0.0: 0.9980217886709782,
 0.1: 0.9991178025469399,
 0.2: 0.9994646074638233,
 0.30000000000000004: 0.9995802957015458,
 0.4: 0.999696012503258,
 0.5: 0.999696012503258,
 0.6000000000000001: 0.999811780992189,
 0.7000000000000001: 0.9998696774783643,
 0.8: 0.9999275862062493,
 0.9: 0.9999855139767939}


#### for etas and phis

In [ ]:
ETAs = [0.1, 0.3, 0.7, 0.9, 0.1, 0.3, 0.7, 0.9]
PHIs = [0.1, 0.3, 0.7, 0.9, 0.9, 0.7, 0.3, 0.1]

for eta, phi in zip(ETAs, PHIs):
    print(f"\n\neta: {eta}, phi: {phi}")
    pos_weight_all, alpha_all = get_pos_weight_alpha(
        split="val2017_person", mask_folder="cmaps_person" + f"_eta{eta}_phi{phi}"
    )
    print("\n\n")



eta: 0.1, phi: 0.1
Calculating pos_weight and alpha for val2017_person...
loading annotations into memory...
Done (t=0.15s)
creating index...
index created!

pos_weight_all for val2017:
{0.0: 3.6053196327234907,
 0.1: 3.607253893433745,
 0.2: 3.622340511677038,
 0.30000000000000004: 3.6814297209445215,
 0.4: 3.841902438222434,
 0.5: 4.202842222364503,
 0.6000000000000001: 5.078715004900064,
 0.7000000000000001: 7.161764223398528,
 0.8: 13.94209510300004,
 0.9: 51.131191496544396}

alpha_all for val2017:
{0.0: 0.722632082070185,
 0.1: 0.7227808106825273,
 0.2: 0.7239353956988905,
 0.30000000000000004: 0.7283664022402045,
 0.4: 0.7397122867954246,
 0.5: 0.7620657766597282,
 0.6000000000000001: 0.8030998000409205,
 0.7000000000000001: 0.8603696004494459,
 0.8: 0.9282747684180679,
 0.9: 0.9804424663159359}





eta: 0.3, phi: 0.3
Calculating pos_weight and alpha for val2017_person...
loading annotations into memory...
Done (t=0.34s)
creating index...
index created!
/mnt/ssd2/xxx/data/coc

## pos_weight for arbitary task and n_classes/target_category_ids


In [ ]:
def get_pos_weight_alpha(
    split="val2017",
    coco_folder=COCO_FOLDER,
    mask_folder="cmaps",
    target_category_ids=None,
    n_classes=80,
    probas=np.linspace(0, 1, 11)[:-1],
):
    if target_category_ids:
        # overwrite n_classes
        n_classes = len(target_category_ids)
    elif n_classes:
        target_category_ids = list(sorted(COCO_ID_TO_CHANNEL.keys()))[:n_classes]

    coco_folder = Path(coco_folder)
    file_name = f"instances_{split}.json"
    annFile = coco_folder / "annotations" / file_name
    coco = COCO(annFile)
    split = split.split("_")[0]

    img_id_to_cat_id = defaultdict(set)
    for cat_id in target_category_ids:
        img_ids = coco.getImgIds(catIds=cat_id)
        for img_id in img_ids:
            img_id_to_cat_id[img_id].add(cat_id)

    area_all = 0
    area_centerness_all = {proba: 0 for proba in probas}

    print(f"Calculating pos_weight and alpha for {split}...")

    for id, cat_ids in img_id_to_cat_id.items():

        w, h = coco.imgs[id]["width"], coco.imgs[id]["height"]
        area = w * h
        area_all += area * len(cat_ids)

        for category_id in cat_ids:
            mask_path = (
                coco_folder
                / mask_folder
                / str(category_id)
                / split
                / coco.loadImgs(id)[0]["file_name"]
            ).with_suffix(".png")
            # print(mask_path)
            mask = (
                cv2.imread(mask_path, cv2.IMREAD_UNCHANGED).astype(np.float32) / 255.0
            )
            for proba in probas:
                area_centerness_all[proba] += np.sum(mask > proba)

    pos_weight_all = {k: area_all / v for k, v in area_centerness_all.items()}

    print(f"\npos_weight_all for {split}:")
    pprint.pprint(pos_weight_all)

    alpha_all = {k: 1 - 1 / v for k, v in pos_weight_all.items()}

    print(f"\nalpha_all for {split}:")
    pprint.pprint(alpha_all)

    return pos_weight_all, alpha_all

### cmaps results

In [8]:
pos_weight_all, alpha_all = get_pos_weight_alpha("val2017", n_classes=1)

loading annotations into memory...
Done (t=0.61s)
creating index...
index created!
Calculating pos_weight and alpha for val2017...

pos_weight_all for val2017:
{0.0: 3.606717387392169,
 0.1: 4.1283694509697035,
 0.2: 5.282034376573399,
 0.30000000000000004: 7.1494939578340055,
 0.4: 10.446969868779867,
 0.5: 16.128149160971073,
 0.6000000000000001: 27.780025203336514,
 0.7000000000000001: 53.39112951806463,
 0.8: 133.78171998496938,
 0.9: 557.5448191029582}

alpha_all for val2017:
{0.0: 0.7227395738031339,
 0.1: 0.7577736169506069,
 0.2: 0.8106790057188671,
 0.30000000000000004: 0.8601299608199182,
 0.4: 0.9042784642283273,
 0.5: 0.9379966051889,
 0.6000000000000001: 0.9640029124278874,
 0.7000000000000001: 0.981270296975799,
 0.8: 0.9925251372152163,
 0.9: 0.998206422218022}


In [31]:
pos_weight_all, alpha_all = get_pos_weight_alpha("val2017", n_classes=80)

loading annotations into memory...
Done (t=0.56s)
creating index...
index created!
Calculating pos_weight and alpha for val2017...

pos_weight_all for val2017:
{0.0: 5.728126992973447,
 0.1: 6.593768345766961,
 0.2: 8.512155056516058,
 0.30000000000000004: 11.615376293530874,
 0.4: 17.08354843145427,
 0.5: 26.48045742944857,
 0.6000000000000001: 45.73752443268081,
 0.7000000000000001: 88.04433363024589,
 0.8: 220.82780443949892,
 0.9: 921.3561475243769}

alpha_all for val2017:
{0.0: 0.8254228648864322,
 0.1: 0.8483416541859594,
 0.2: 0.8825209370176476,
 0.30000000000000004: 0.9139072230869572,
 0.4: 0.9414641516654234,
 0.5: 0.9622363018968126,
 0.6000000000000001: 0.9781361144398653,
 0.7000000000000001: 0.988642085654261,
 0.8: 0.995471584737537,
 0.9: 0.9989146433735892}


In [ ]:
pos_weight_all, alpha_all = get_pos_weight_alpha("val2017", n_classes=80)

Calculating pos_weight and alpha for val2017...
loading annotations into memory...
Done (t=0.80s)
creating index...
index created!

pos_weight_all for val2017:
{0.0: 5.728126992973447,
 0.1: 6.593768345766961,
 0.2: 8.512155056516058,
 0.30000000000000004: 11.615376293530874,
 0.4: 17.08354843145427,
 0.5: 26.48045742944857,
 0.6000000000000001: 45.73752443268081,
 0.7000000000000001: 88.04433363024589,
 0.8: 220.82780443949892,
 0.9: 921.3561475243769}

alpha_all for val2017:
{0.0: 0.8254228648864322,
 0.1: 0.8483416541859594,
 0.2: 0.8825209370176476,
 0.30000000000000004: 0.9139072230869572,
 0.4: 0.9414641516654234,
 0.5: 0.9622363018968126,
 0.6000000000000001: 0.9781361144398653,
 0.7000000000000001: 0.988642085654261,
 0.8: 0.995471584737537,
 0.9: 0.9989146433735892}


#### for etas and phis

In [ ]:
ETAs = [0.5, 0.7, 0.1, 0.3, 0.7, 0.9]
PHIs = [0.5, 0.7, 0.9, 0.7, 0.3, 0.1]
TARGET = [35]

for eta, phi in zip(ETAs, PHIs):

    if eta == 0.5 and phi == 0.5:
        mask_folder = "cmaps"
    else:
        mask_folder = "cmaps" + f"_{eta}_{phi}"
    print(f"\n\neta: {eta}, phi: {phi}")
    pos_weight_all, alpha_all = get_pos_weight_alpha(
        split="val2017", mask_folder=mask_folder, target_category_ids=TARGET
    )
    print("\n\n")



eta: 0.5, phi: 0.5
loading annotations into memory...
Done (t=0.60s)
creating index...
index created!
Calculating pos_weight and alpha for val2017...

pos_weight_all for val2017:
{0.0: 18.832373124071527,
 0.1: 21.83124077980194,
 0.2: 28.3728618017323,
 0.30000000000000004: 38.842043613408066,
 0.4: 57.1817502398448,
 0.5: 88.41995464852607,
 0.6000000000000001: 151.73945210489055,
 0.7000000000000001: 288.90813493657555,
 0.8: 705.582332294142,
 0.9: 2755.1167106495277}

alpha_all for val2017:
{0.0: 0.9468999475843116,
 0.1: 0.95419408314505,
 0.2: 0.9647550533679706,
 0.30000000000000004: 0.9742547016847793,
 0.4: 0.9825119029094848,
 0.5: 0.9886903357508489,
 0.6000000000000001: 0.9934097560909292,
 0.7000000000000001: 0.9965386921340255,
 0.8: 0.9985827309525331,
 0.9: 0.9996370389696616}





eta: 0.7, phi: 0.7
loading annotations into memory...
Done (t=0.56s)
creating index...
index created!
Calculating pos_weight and alpha for val2017...

pos_weight_all for val2017:
{0.0: 18.

In [ ]:
ETAs = [0.5, 0.7, 0.1, 0.3, 0.7, 0.9]
PHIs = [0.5, 0.7, 0.9, 0.7, 0.3, 0.1]
TARGET = [44]

for eta, phi in zip(ETAs, PHIs):

    if eta == 0.5 and phi == 0.5:
        mask_folder = "cmaps"
    else:
        mask_folder = "cmaps" + f"_{eta}_{phi}"

    print(f"\n\neta: {eta}, phi: {phi}")
    pos_weight_all, alpha_all = get_pos_weight_alpha(
        split="val2017", mask_folder=mask_folder, target_category_ids=TARGET
    )

    print("\n\n")



eta: 0.5, phi: 0.5
loading annotations into memory...
Done (t=0.59s)
creating index...
index created!
Calculating pos_weight and alpha for val2017...

pos_weight_all for val2017:
{0.0: 27.62880522108167,
 0.1: 31.973630571985176,
 0.2: 41.35031565693367,
 0.30000000000000004: 56.409871994359165,
 0.4: 82.78952724862106,
 0.5: 127.70154041873946,
 0.6000000000000001: 218.72840577148014,
 0.7000000000000001: 416.06515703694436,
 0.8: 1013.6886134557857,
 0.9: 3864.69704344036}

alpha_all for val2017:
{0.0: 0.9638058905552322,
 0.1: 0.9687242273676551,
 0.2: 0.9758163877563455,
 0.30000000000000004: 0.9822726064668256,
 0.4: 0.9879211775542944,
 0.5: 0.9921692408977922,
 0.6000000000000001: 0.9954281201087125,
 0.7000000000000001: 0.9975965302955874,
 0.8: 0.9990135037656279,
 0.9: 0.9997412475056234}





eta: 0.7, phi: 0.7
loading annotations into memory...
Done (t=0.58s)
creating index...
index created!
Calculating pos_weight and alpha for val2017...

pos_weight_all for val2017:
{0.0

### gmaps_kp results

In [8]:
pos_weight_all, alpha_all = get_pos_weight_alpha(
    split="val2017", mask_folder="gmaps_kp", n_classes=80
)

Calculating pos_weight and alpha for val2017...
loading annotations into memory...
Done (t=0.56s)
creating index...
index created!

pos_weight_all for val2017:
{0.0: 817.9442287341551,
 0.1: 1824.244637718209,
 0.2: 3001.1178175576115,
 0.30000000000000004: 3826.4083116008924,
 0.4: 5280.575006560302,
 0.5: 5280.575006560302,
 0.6000000000000001: 8525.738462222862,
 0.7000000000000001: 12311.391255536793,
 0.8: 22157.00689818434,
 0.9: 110771.00836704925}

alpha_all for val2017:
{0.0: 0.9987774227571143,
 0.1: 0.999451827907659,
 0.2: 0.9996667908223563,
 0.30000000000000004: 0.9997386583138637,
 0.4: 0.9998106266838824,
 0.5: 0.9998106266838824,
 0.6000000000000001: 0.9998827081074054,
 0.7000000000000001: 0.9999187744114988,
 0.8: 0.99995486755027,
 0.9: 0.9999909723670954}


### emaps results

In [5]:
pos_weight_all, alpha_all = get_pos_weight_alpha(
    split="val2017", mask_folder="emaps", n_classes=1
)

Calculating pos_weight and alpha for val2017...
loading annotations into memory...
Done (t=0.26s)
creating index...
index created!

pos_weight_all for val2017:
{0.0: 4.524946817714009,
 0.1: 5.47351800388535,
 0.2: 6.855364088932975,
 0.30000000000000004: 8.794760773555128,
 0.4: 11.914149416715203,
 0.5: 16.893427081918016,
 0.6000000000000001: 26.416160857037642,
 0.7000000000000001: 46.11852285013789,
 0.8: 104.54111036126541,
 0.9: 398.2845297267721}

alpha_all for val2017:
{0.0: 0.7790029274851904,
 0.1: 0.8173021447467324,
 0.2: 0.8541288271451024,
 0.30000000000000004: 0.8862959407597658,
 0.4: 0.9160661860932322,
 0.5: 0.9408053797994395,
 0.6000000000000001: 0.9621443855747273,
 0.7000000000000001: 0.9783167383038373,
 0.8: 0.990434385128068,
 0.9: 0.9974892321308939}
